<a href="https://colab.research.google.com/github/dkanzariya/Machine-learning-in-Communication-/blob/master/TP_communication.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import torch
import math
from torch import nn
from torch.autograd import Variable

#An Introduction to deep learning for the physical layer

https://arxiv.org/pdf/1702.00832.pdf

In [ ]:
class RadioTransNets(nn.Module):
  def __init__(self,in_channel,compossed_dim):
    
    super(RadioTransNets,self).__init__()
    self.in_channel = in_channel

    self.encoder = nn.Sequential(
        nn.Linear(in_channel , in_channel),
        nn.ReLU(inplace = True),
        nn.Linear(in_channel,compossed_dim))
    
    self.decoder = nn.Sequential(
        nn.Linear(compossed_dim , compossed_dim),
        nn.ReLU(inplace = True),
        nn.Linear(compossed_dim ,in_channel))

  def forward(self,x):
    x = self.encoder(x)

    x = (self.in_channel ** 2) * (x/x.norm(dim = -1)[: , None])

    noise = Variable(torch.randn(*x.size()) / ((2* communication_rate * train_snr) ** 0.5))

    x+=noise
    x = self.decoder(x)

    return x

In [ ]:
num_epochs = 100
batch_size = 256
channel_size = 4
train_snr = 5.01187
communication_rate = 1

train_labels = (torch.rand(10000) * channel_size).long()
train_data = torch.sparse.torch.eye(channel_size).index_select(dim=0, index=train_labels)

test_labels = (torch.rand(1500) * channel_size).long()
test_data = torch.sparse.torch.eye(channel_size).index_select(dim=0, index=test_labels)

#Model 
~accuracy 

In [ ]:
model = RadioTransNets(channel_size, compossed_dim=int(math.log2(channel_size)))
optimizer = torch.optim.Adam(model.parameters())
loss_fn = nn.CrossEntropyLoss()

def init_weights(m):
    if type(m) == nn.Linear:
        torch.nn.init.normal_(m.weight, std=0.01)

model.apply(init_weights)

RadioTransNets(
  (encoder): Sequential(
    (0): Linear(in_features=4, out_features=4, bias=True)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=4, out_features=2, bias=True)
  )
  (decoder): Sequential(
    (0): Linear(in_features=2, out_features=2, bias=True)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=2, out_features=4, bias=True)
  )
)

In [ ]:
for t in range(1000):

  train_data = Variable(train_data)
  y_train = model(train_data)
  loss = loss_fn(y_train, train_labels)
  if t%100 == 99:
    pred = y_train.data.numpy()
    label = train_labels.data.numpy()
    pred_output = np.argmax(pred,axis=1)
    no_errors = (pred_output != label)
    no_errors =  no_errors.astype(int).sum()
    ber = no_errors / 10000
    ber = np.double(ber)
    # pred_label = torch.max(y_train, 1)[1].data.squeeze()
    # accuracy = sum(pred_label == train_labels) / float(train_labels.size(0))
    print("epoch : {}    accuracy : {}     loss : {}".format(t, ber, loss.item()))

  optimizer.zero_grad()
  loss.backward()
  optimizer.step()

epoch : 99    accuracy : 0.496     loss : 1.36151921749115
epoch : 199    accuracy : 0.4929     loss : 0.9166845679283142
epoch : 299    accuracy : 0.4999     loss : 0.8443883657455444
epoch : 399    accuracy : 0.249     loss : 0.6221941113471985
epoch : 499    accuracy : 0.249     loss : 0.48520106077194214
epoch : 599    accuracy : 0.249     loss : 0.45691177248954773
epoch : 699    accuracy : 0.249     loss : 0.44023677706718445
epoch : 799    accuracy : 0.249     loss : 0.4278075098991394
epoch : 899    accuracy : 0.249     loss : 0.41805142164230347
epoch : 999    accuracy : 0.249     loss : 0.4099169969558716


In [ ]:
y_test = model(test_data)
loss = loss_fn(y_test, test_labels)

pred = y_test.data.numpy()
label = test_labels.data.numpy()
pred_output = np.argmax(pred,axis=1)
no_errors = (pred_output != label)
no_errors =  no_errors.astype(int).sum()
ber_test = no_errors / 1500
ber_test = np.double(ber)
# pred_labels = torch.max(y_test, 1)[1].data.squeeze()
# accuracy = sum(pred_labels == test_labels) / float(test_labels.size(0))

print("testing accuracy : {}  loss : {}".format(ber_test, loss.item()))

testing accuracy : 0.25133333333333335  loss : 0.41300496459007263
